In [3]:
# Imports and global tolerance
import mercury as mr
import random
import csv

import pandas as pd
import numpy as np
import joblib
import os

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

tolerance = 1e-7

In [4]:
# User interface inputs
show_code = mr.Checkbox(value=False, label="Show source code")
app = mr.App(
    title="Loan House Calculation",
    description="Actuarial Mathematics",
    show_code=show_code.value,
    continuous_update=True
)

admin_toggle = mr.Checkbox(value=False, label="Enable Admin Mode (Simulate Random Contracts)")
details_toggle = mr.Checkbox(value=False, label="Show detailed calculations")
_ = mr.Note(text="---")

if admin_toggle.value == True:
    num_contracts = mr.Numeric(label="Number of Random Contracts", value=50, min=1, max=30000)
else:
    age = mr.Numeric(label="Age (Years)", value=22, min=18, max=100)
    amount = mr.Numeric(label="Loan Amount", min=0, value=150000, max=999999999999)
    interest_rate = mr.Numeric(label="Yearly Interest Rate (%)", value=2.5, min=0.0, max=100.0, step=0.1)
    maturity = mr.Numeric(label="Maturity (Years)", value=25, min=1, max=50)
    single_premium_model = mr.File(label="Upload Single Premium Model", max_file_size="50MB")
    annual_premium_model = mr.File(label="Upload Annual Premium Model", max_file_size="50MB")
    scaler = mr.File(label="Upload Scaler", max_file_size="50MB")

mortality_table = mr.Select(value = "TD88-90 (France)", choices = ["TD88-90 (France)", "TH00-02 (France)"], label = "Select Mortality Table")
currency = mr.Select(value = "EUR", choices = ["EUR", "USD", "GBP"], label = "Select Currency")

mercury.Checkbox

mercury.App

mercury.Checkbox

mercury.Checkbox

---

...

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Select

mercury.Select

In [5]:
_ = mr.Note(text="---")
_ = mr.Note("Please fill out the inputs above and click _Apply_ or hit Enter to calculate the result.")

---

Please fill out the inputs above and click _Apply_ or hit Enter to calculate the result.

In [6]:
# Load mortality table
def load_mortality_data(filename):
    with open(filename, "r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        return {int(row[0]): int(row[1]) for row in reader}

if mortality_table.value == "TD88-90 (France)":
    mortality_data = load_mortality_data("TD88-90.csv")
elif mortality_table.value == "TH00-02 (France)":
    mortality_data = load_mortality_data("TH00-02.csv")

In [7]:
# Calculation helper functions
def annuity_factor(x, m, tech_rate):
    act_sum = 0
    for i in range(0, m):
        act_sum += nex(x, i, tech_rate)
    return act_sum

def nex(x, n, tech_rate):
    return npx(x, n) * techDF(n, tech_rate)

def npx(x ,n):
    return mortality_data[x+n] / mortality_data[x]

def techDF(n, tech_rate):
    return 1 / ((1 + tech_rate) ** n)

def n_1qx(x, n):
    return (mortality_data[x+n] - mortality_data[x+n+1]) / mortality_data[x]

In [8]:
# Core logic
def calculate_contract(p_age, p_maturity, p_interest, p_amount):
    number_of_payments = int(p_maturity * 12) # input is in years
    periodic_rate = p_interest / 12 / 100 # input is yearly + should be divided by 100
    monthly_reimbursed_amount = (p_amount * periodic_rate) / (1 - (1 + periodic_rate) ** -number_of_payments)

    remaining_amount = []
    interest = []
    amortization = []

    for i in range(0, number_of_payments + 1):
        # Special cases on the 0 month
        if i == 0:
            remaining_amount.append(p_amount)
            interest.append(0)
            amortization.append(0)
            continue

        # Remaining amount (end of the year) Ck
        # Using tolerance for the last element (not perfect zero)
        calculated_value = (1 + periodic_rate) * remaining_amount[-1] - monthly_reimbursed_amount
        remaining_amount.append(0 if calculated_value < tolerance else calculated_value)

        # Interest Ik
        interest.append(remaining_amount[i-1] * periodic_rate)
        # Amortization Ak
        amortization.append(monthly_reimbursed_amount - interest[i])

    total_reimbursement_amount = sum(amortization)
    cost_of_loan = sum(interest)

    # Yearly payment calculations
    k = 0
    discounted_amount_in_risks = []

    for i in range(1, len(remaining_amount)):
        if i % 12 == 0:
            benefit_akprime = remaining_amount[i]
            discount_factor = (1 / (1 + p_interest / 100)) ** (k+1)
            probability = (mortality_data[p_age+k] - mortality_data[p_age+k+1]) / mortality_data[p_age]
            discounted_amount_in_risk = benefit_akprime * discount_factor * probability
            discounted_amount_in_risks.append(discounted_amount_in_risk)
            k += 1
    total_discounted_amount_in_risk = sum(discounted_amount_in_risks)
    single_premium = total_discounted_amount_in_risk

    annuity_factor_calc = annuity_factor(int(p_age), int(p_maturity), p_interest / 100)
    annual_premium = single_premium / annuity_factor_calc

    # Monthly payment calculations
    monthly_annuity_factor = annuity_factor_calc - ((12 - 1 )/( 2 * 12)) * (1 - nex(p_age, p_maturity, p_interest / 100))
    monthly_annual_premium = single_premium / monthly_annuity_factor
    monthly_premium = monthly_annual_premium / 12

    return single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, number_of_payments, periodic_rate, monthly_reimbursed_amount, remaining_amount, interest, amortization, total_reimbursement_amount, cost_of_loan


In [9]:
# Simulation helper functions
def evaluate_model(y_test, y_pred, target_name):
    print(f"{target_name} Prediction Performance:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
    print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")
    print(f"R² Score: {r2_score(y_test, y_pred):.4f}\n")

def plot_learning_curve():
    sizes = list(range(50, len(X_scaled), 50))

    r2_scores_y1, r2_scores_y2 = [], []
    mae_scores_y1, mae_scores_y2 = [], []
    rmse_scores_y1, rmse_scores_y2 = [], []

    hyperparameters = {
        "n_estimators": 200,
        "max_depth": 10,
        "min_samples_split": 10,
        "min_samples_leaf": 2,
        "random_state": 42
    }

    for size in sizes:
        X_sample, y1_sample, y2_sample = X_scaled[:size], y1[:size], y2[:size]

        if size < 50:
            continue

        model1_cv = RandomForestRegressor(n_estimators=hyperparameters["n_estimators"],
                                          max_depth=hyperparameters["max_depth"],
                                          min_samples_split=hyperparameters["min_samples_split"],
                                          min_samples_leaf=hyperparameters["min_samples_leaf"],
                                          random_state=hyperparameters["random_state"])

        model2_cv = RandomForestRegressor(n_estimators=hyperparameters["n_estimators"],
                                          max_depth=hyperparameters["max_depth"],
                                          min_samples_split=hyperparameters["min_samples_split"],
                                          min_samples_leaf=hyperparameters["min_samples_leaf"],
                                          random_state=hyperparameters["random_state"])

        model1_cv.fit(X_sample, y1_sample)
        model2_cv.fit(X_sample, y2_sample)

        y1_pred_sample = model1_cv.predict(X_sample)
        y2_pred_sample = model2_cv.predict(X_sample)

        r2_scores_y1.append(r2_score(y1_sample, y1_pred_sample))
        r2_scores_y2.append(r2_score(y2_sample, y2_pred_sample))

        mae_scores_y1.append(mean_absolute_error(y1_sample, y1_pred_sample))
        mae_scores_y2.append(mean_absolute_error(y2_sample, y2_pred_sample))

        rmse_scores_y1.append(np.sqrt(mean_squared_error(y1_sample, y1_pred_sample)))
        rmse_scores_y2.append(np.sqrt(mean_squared_error(y2_sample, y2_pred_sample)))

    if not sizes:
        print("Error: Not enough data points to plot the learning curve.")
        return

    print("RandomForestRegressor Model Hyperparameters:")
    for param, value in hyperparameters.items():
        print(f"{param}: {value}")

    print("\nTraining and Evaluation Results:")

    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.set_xlabel("Dataset Size")
    ax1.set_ylabel("R² Score", color="tab:blue")
    ax1.plot(sizes, r2_scores_y1, marker="o", label="Single Premium R²", color="blue")
    ax1.plot(sizes, r2_scores_y2, marker="o", label="Annual Premium R²", color="cyan")
    ax1.axhline(y=0.90, color="r", linestyle="--", label="90% Threshold")
    ax1.legend(loc="upper left")
    ax1.tick_params(axis="y", labelcolor="tab:blue")
    plt.title("Learning Curve: R² vs. Dataset Size")
    plt.grid()
    plt.show()

    fig, ax2 = plt.subplots(figsize=(12, 6))
    ax2.set_xlabel("Dataset Size")
    ax2.set_ylabel("MAE", color="tab:orange")
    ax2.plot(sizes, mae_scores_y1, marker="s", linestyle="dashed", color="orange", label="Single Premium MAE")
    ax2.plot(sizes, mae_scores_y2, marker="s", linestyle="dashed", color="red", label="Annual Premium MAE")
    ax2.legend(loc="upper left")
    ax2.tick_params(axis="y", labelcolor="tab:orange")
    plt.title("Learning Curve: MAE vs. Dataset Size")
    plt.grid()
    plt.show()

    fig, ax3 = plt.subplots(figsize=(12, 6))
    ax3.set_xlabel("Dataset Size")
    ax3.set_ylabel("RMSE", color="tab:green")
    ax3.plot(sizes, rmse_scores_y1, marker="^", linestyle="dotted", color="brown", label="Single Premium RMSE")
    ax3.plot(sizes, rmse_scores_y2, marker="^", linestyle="dotted", color="purple", label="Annual Premium RMSE")
    ax3.legend(loc="upper left")
    ax3.tick_params(axis="y", labelcolor="tab:green")
    plt.title("Learning Curve: RMSE vs. Dataset Size")
    plt.grid()
    plt.show()

def save_trained_models(model1, model2, scaler, output_dir):
    model1_path = os.path.join(output_dir.path, "single_premium_model.pkl")
    model2_path = os.path.join(output_dir.path, "annual_premium_model.pkl")
    scaler_path = os.path.join(output_dir.path, "scaler.pkl")

    joblib.dump(model1, model1_path)
    joblib.dump(model2, model2_path)
    joblib.dump(scaler, scaler_path)

    print("✅ Models and scaler saved successfully!")

In [16]:
# If admin simulation
if admin_toggle.value == True:
    random_contracts = []

    ages = list(mortality_data.keys())
    lx_values = list(mortality_data.values())

    # Normalize lx values to create a probability distribution
    total_lx = sum(lx_values)
    probabilities = [lx / total_lx for lx in lx_values]

    for _ in range(int(num_contracts.value)):
        age = random.choices(ages, probabilities)[0]
        loan_amount = random.randint(1000, 200000)
        interest_rate = random.uniform(0.5, 12.0)
        maturity = random.randint(1, 100 - min(age, 99)) # upper do not to overflow the mortality table

        single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, number_of_payments, periodic_rate, monthly_reimbursed_amount, remaining_amount, interest, amortization, total_reimbursement_amount, cost_of_loan = calculate_contract(age, maturity, interest_rate, loan_amount)

        contract = {
            'age': age,
            'loan_amount': loan_amount,
            'interest_rate': interest_rate,
            'maturity': maturity,

            'annuity_factor_calc': annuity_factor_calc,
            'monthly_premium': monthly_premium,
            'monthly_annual_premium': monthly_annual_premium,
            'monthly_annuity_factor': monthly_annuity_factor,
            'number_of_payments': number_of_payments,
            'periodic_rate': periodic_rate,
            'monthly_reimbursed_amount': monthly_reimbursed_amount,
            'final_remaining_amount': remaining_amount[-1],
            'total_interest_paid': sum(interest),
            'total_amortization': sum(amortization),
            'total_reimbursement_amount': total_reimbursement_amount,
            'cost_of_loan': cost_of_loan,

            'single_premium': single_premium,
            'annual_premium': annual_premium
        }
        random_contracts.append(contract)

    df = pd.DataFrame(random_contracts)
    df = shuffle(df, random_state=42).reset_index(drop=True)

    X = df[['age', 'loan_amount', 'interest_rate', 'maturity',
        'annuity_factor_calc', 'monthly_premium', 'monthly_annual_premium', 'monthly_annuity_factor',
        'number_of_payments', 'periodic_rate', 'monthly_reimbursed_amount', 'final_remaining_amount',
        'total_interest_paid', 'total_amortization', 'total_reimbursement_amount', 'cost_of_loan']]

    y1 = df['single_premium']
    y2 = df['annual_premium']

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    model1 = RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=2, min_samples_leaf=2, random_state=42)
    model2 = RandomForestRegressor(n_estimators=300, max_depth=20, min_samples_split=2, min_samples_leaf=2, random_state=42)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores1 = cross_val_score(model1, X_scaled, y1, cv=cv, scoring='neg_mean_squared_error')
    print(f"Cross-validated MSE for Single Premium: {-cv_scores1.mean():.2f}")

    cv_scores2 = cross_val_score(model2, X_scaled, y2, cv=cv, scoring='neg_mean_squared_error')
    print(f"Cross-validated MSE for Annual Premium: {-cv_scores2.mean():.2f}")

    model1.fit(X_scaled, y1)
    model2.fit(X_scaled, y2)

    y1_pred = model1.predict(X_scaled)
    y2_pred = model2.predict(X_scaled)

    evaluate_model(y1, y1_pred, "Single Premium")
    evaluate_model(y2, y2_pred, "Annual Premium")

    plot_learning_curve()
    X_original = scaler.inverse_transform(X_scaled)

    print("\nExact Predictions for the first 10 contracts:\n")
    for i in range(min(10, len(y1_pred))):
        age = int(X_original[i][0])
        loan_amount = int(X_original[i][1])
        interest_rate = X_original[i][2]
        maturity = int(X_original[i][3])

        print(f"    Age: {age}")
        print(f"    Loan Amount: {loan_amount}")
        print(f"    Interest Rate: {interest_rate:.2f}%")
        print(f"    Maturity: {maturity}")
        print(f"    Actual Single Premium: {y1.iloc[i]:,.2f}")
        print(f"    Actual Annual Premium: {y2.iloc[i]:,.2f}")
        print(f"    Predicted Single Premium: {y1_pred[i]:,.2f}")
        print(f"    Predicted Annual Premium: {y2_pred[i]:,.2f}")
        print("-" * 50)

    # Save the models
    output_dir = mr.OutputDir()
    save_trained_models(model1, model2, scaler, output_dir)

    mr.Stop()
else:
    if age and age.value + maturity.value > 100:
        mr.Markdown(text=f'#<font color="red">ERROR: With these values no house loan is available. Please check your inputs.</font>')
        mr.Stop()

In [11]:
# Stop plotting if it is simulation
if admin_toggle.value == True:
    mr.Stop()

single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, number_of_payments, periodic_rate, monthly_reimbursed_amount, remaining_amount, interest, amortization, total_reimbursement_amount, cost_of_loan = calculate_contract(age.value, maturity.value, interest_rate.value, amount.value)

colors = ["#F8F9FA", "green", "#FCE700", "#00F5FF"]
mr.Markdown(text=f'#<font color="black">Yearly Payment</font>')
mr.NumberBox([
    mr.NumberBox(data=float(f"{single_premium:.2f}"), title=f"Single premium ({currency.value})", background_color=colors[0], border_color=colors[1], data_color=colors[1], title_color=colors[1]),
    mr.NumberBox(data=float(f"{annual_premium:.2f}"), title=f"Annual premium ({currency.value})", background_color=colors[0], border_color=colors[1], data_color=colors[1], title_color=colors[1]),
    mr.NumberBox(data=float(f"{annuity_factor_calc:.3f}"), title="Annuity factor")
])

#<font color="black">Yearly Payment</font>

In [17]:
if single_premium_model.filepath and annual_premium_model.filepath and scaler.filepath:
    single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, number_of_payments, periodic_rate, monthly_reimbursed_amount, remaining_amount, interest, amortization, total_reimbursement_amount, cost_of_loan = calculate_contract(age.value, maturity.value, interest_rate.value, amount.value)

    model1 = joblib.load(single_premium_model.filepath)
    model2 = joblib.load(annual_premium_model.filepath)
    scaler = joblib.load(scaler.filepath)

    input_data = np.array([[age.value, amount.value, interest_rate.value, maturity.value, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, number_of_payments, periodic_rate, monthly_reimbursed_amount, remaining_amount[-1], sum(interest), sum(amortization), total_reimbursement_amount, cost_of_loan]])
    column_names = scaler.feature_names_in_
    input_data_df = pd.DataFrame(input_data, columns=column_names)

    input_scaled = scaler.transform(input_data_df)
    single_premium_prediction = model1.predict(input_scaled)
    annual_premium_prediction = model2.predict(input_scaled)

    mr.Markdown(text=f'#<font color="black">Predicted values by model</font>')
    mr.Markdown(text=f'###<font color="black">Predicted Monthly Premium: {currency.value} {single_premium_prediction[0]:.2f} </font>')
    mr.Markdown(text=f'###<font color="black">Monthly Premium Diff: {(abs(single_premium - single_premium_prediction[0]) / single_premium) * 100:.2f}%</font>')
    mr.Markdown(text=f'###<font color="black">Predicted Annual Premium: {currency.value} {annual_premium_prediction[0]:.2f} </font>')
    mr.Markdown(text=f'###<font color="black">Predicted Annual Premium Diff: {(abs(annual_premium - annual_premium_prediction[0]) / annual_premium) * 100:.2f}%</font>')


NameError: name 'single_premium_model' is not defined

In [12]:
mr.Markdown(text=f'#<font color="black">Monthly Payment</font>')
mr.NumberBox([
    mr.NumberBox(data=float(f"{monthly_premium:.2f}"), title=f"Monthly premium ({currency.value})"),
    mr.NumberBox(data=float(f"{monthly_annual_premium:.2f}"), title=f"Annual premium ({currency.value})"),
    mr.NumberBox(data=float(f"{monthly_annuity_factor:.3f}"), title="Annuity factor")
])

#<font color="black">Monthly Payment</font>

In [13]:
# Balance sheet for year 1
# Sum of Assets and Liability should be equal

# Assets
balance_sheet_annual_premium = annual_premium
balance_sheet_interest = balance_sheet_annual_premium * interest_rate.value / 100
balance_sheet_sum_of_assets = balance_sheet_annual_premium + balance_sheet_interest

# Liability
balance_sheet_claims = remaining_amount[12] * (1 - mortality_data[age.value + 1 ] / mortality_data[age.value])
balance_sheet_recurrence = (0 + balance_sheet_annual_premium - remaining_amount[12] * (1 / (1 + interest_rate.value / 100)) * n_1qx(age.value+12/12-1, 0)) / ((1 / (1 + interest_rate.value / 100)) * (1 -  n_1qx(age.value+12/12-1, 0)))
balance_sheet_reserves = balance_sheet_recurrence * mortality_data[age.value + 1 ] / mortality_data[age.value]
balance_sheet_sum_of_liabilities = balance_sheet_claims + balance_sheet_reserves

# Validation check: assets should be equal to liabilities
if balance_sheet_sum_of_assets - balance_sheet_sum_of_liabilities > tolerance:
    mr.Markdown(text=f'#<font color="red">ERROR: The difference between the assets and the liabilities are greater than the tolerance.</font>')

mr.Markdown(text=f'#<font color="black">Balance Sheet for Year 1</font>')

df = pd.DataFrame(
    {
        "Assets": ["Annual Premium*", "Interest (financial income)", "Opening", "Sum of Assets"],
        "Assets Value": [
            f"{float(balance_sheet_annual_premium):.2f} {currency.value}",
            f"{float(balance_sheet_interest):.2f} {currency.value}",
            f"{float(0):.2f} {currency.value}",
            f"{float(balance_sheet_sum_of_assets):.2f} {currency.value}"
        ],
        "Liability": ["Claims", "Premium Reserves", "", "Sum of Liabilities"],
        "Liability Value": [
            f"{float(balance_sheet_claims):.2f} {currency.value}",
            f"{float(balance_sheet_reserves):.2f} {currency.value}",
            "",
            f"{float(balance_sheet_sum_of_liabilities):.2f} {currency.value}"
        ]
    }
)

mr.Table(data=df, width="150px", text_align="left")

#<font color="black">Balance Sheet for Year 1</font>

In [14]:
print("* We are using annual premium with yearly payment for balance sheet calculations.")

* We are using annual premium with yearly payment for balance sheet calculations.


In [15]:
if details_toggle.value == False:
    mr.Stop()

# Detailed calculation
mr.Markdown(text=f'#<font color="black">Detailed calculation</font>')

mr.Markdown(text=f'###<font color="black">Number of payments / Durations (month): {number_of_payments:.0f}</font>')
mr.Markdown(text=f'###<font color="black">Periodic rate: {periodic_rate * 100:.4f} %</font>')
mr.Markdown(text=f'###<font color="black">Cost of loan: {cost_of_loan:.2f} {currency.value}</font>')
mr.Markdown(text=f'###<font color="black">Total reimbursement amount: {total_reimbursement_amount:.2f} {currency.value}</font>')
mr.Markdown(text=f'#') # spacer
mr.Markdown(text=f'#') # spacer

num_rows = len(amortization)

remaining_amount = remaining_amount
interest[0] = pd.NA
amortization[0] = pd.NA
monthly_reimbursed_amount_array = [monthly_reimbursed_amount] * (num_rows)
monthly_reimbursed_amount_with_insurance_array = [monthly_reimbursed_amount + monthly_premium] * (num_rows)
beginning_remaining_amount = [pd.NA] + remaining_amount[:-1]

detailed_calc = pd.DataFrame({
    "Remaining amount (beginning of the year) (Ck-1)": beginning_remaining_amount,
    "Interest (Ik)": interest,
    "Amortization (Ak)": amortization,
    "Monthly reimbursed amount (Ek)": monthly_reimbursed_amount_array,
    "Monthly reimbursed amount with insurance": monthly_reimbursed_amount_with_insurance_array,
    "Remaining amount (end of the year) (Ck)": remaining_amount,
})

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
pd.set_option("display.expand_frame_repr", False)

detailed_calc.index.name = "Month"
styled_table = detailed_calc.style.set_table_attributes('style="display:inline"') \
                                   .set_caption("Detailed Calculation Overview") \
                                   .set_table_styles([
                                       {'selector': 'thead th', 'props': [('background-color', '#D3D3D3'), ('color', 'black'), ('font-weight', 'bold')]},
                                       {'selector': 'index', 'props': [('text-align', 'center'), ('font-weight', 'bold')]},
                                       {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('border', '1px solid black')]},
                                       {'selector': 'thead', 'props': [('border', '1px solid black')]},
                                       {'selector': 'tbody', 'props': [('border', '1px solid black')]},
                                   ])

# Display the styled DataFrame
display(styled_table)

StopExecution: 

In [ ]:
# Mortality table details
mr.Markdown(text=f'#<font color="black">Mortality table details</font>')
mr.Markdown(text=f'###<font color="black">Chosen table: {mortality_table.value}</font>')

df_mortality_data = pd.DataFrame(list(mortality_data.items()), columns=["Age (x)", "lx"])
df_mortality_data